In [3]:
'''
Generates a basis of vanishing cycles
-van_beta: vanishing cycles
'''

# generate all betas corresponding to vanishing cycles
van_beta = []
for b0 in range(5):
    for b1 in range(5):
        for b2 in range(5):
            for b3 in range(5):
                for b4 in range(5):
                    for b5 in range(5):
                        beta_p = [b0,b1,b2,b3,b4,b5]
                        if 0 in beta_p:
                            van_beta.append(beta_p)
                        

In [5]:
# for L a set of betas, find Z(beta,beta') for all beta in L

def Z(L,beta_p):
    '''
    Generates the exponents appearing in the formula
    -Zvec: exponents ( integers ) 
    '''
    Zvec = []
    for beta in L:
        z = vector(beta_p)*vector(beta) + sum(beta_p) 
        Zvec.append(z%6)
    
    return (Zvec)

In [6]:
# generate the set of all generators of our lattice (in terms of powers of zeta_6)
# note that duplicates are not recorded
# note that we need to make sure none of the beta in L are cc or appear twice

def Z_set(L,all_beta_p):
    '''
    Remove the duplicates from the previous list
    -Zs: exponents without duplicates
    '''
    Zs = []
    for beta_p in all_beta_p:
        if not Z(L,beta_p) in Zs:
            Zs.append(Z(L,beta_p))
    return Zs    

In [7]:
# make a list of generators of A2 embedded into ZZ^3, L[i] = zeta_6^i

LZZ = [[1,-1,0],[1,0,-1],[0,1,-1],[-1,1,0],[-1,0,1],[0,-1,1]]

In [8]:
#returns the span of the generators.
#besides not using sagemath, one needs to implement a function to generate the span by hand to be able to scale up
def L_cond(L):
    
    '''
    Find a basis in ZZ^3 of the lattice
    Zs: exponents without duplicates
    gensZZ: result, generators in ZZ^3
    '''
    gensZZ = []
    Zs = Z_set(L,van_beta)
    for entry in Zs:
        ZZvec = []
        for k in entry:
            ZZvec =  ZZvec + LZZ[k]
        
        gensZZ.append(ZZvec)
        
    return span(gensZZ,ZZ)  # Uses row echelon form, extremely slow given the size of the matrix

In [9]:
# generate all betas corresponding to residues = h22
h22_beta = []
for b0 in range(5):
    for b1 in range(5):
        for b2 in range(5):
            for b3 in range(5):
                for b4 in range(5):
                    for b5 in range(5):
                        beta = [b0,b1,b2,b3,b4,b5]
                        if sum(beta) == 12:
                            h22_beta.append(beta)

In [10]:
# generate all betas corresponding to residues = h31
h31_beta = []
for b0 in range(5):
    for b1 in range(5):
        for b2 in range(5):
            for b3 in range(5):
                for b4 in range(5):
                    for b5 in range(5):
                        beta = [b0,b1,b2,b3,b4,b5]
                        if sum(beta) == 6:
                            h31_beta.append(beta)

In [11]:
# here is how to clean up and delete bar(w) if already w is in the list
def nocc(beta_list):
    '''
    Clears the previous list of complex conjugate. TODO: match the convention of the paper, here it is listed as they appear
    cleaned_version: result, all forms without complex conjugates
    '''
    cleaned_version = []
    for beta in beta_list:
        beta_cc = [4-i for i in beta]
        if not beta_cc in cleaned_version:
            cleaned_version.append(beta)
    return cleaned_version     

In [12]:
# find all even (2,2) and (3,1) forms for an arbitrary group actions; define matrix of group action as matrix of weights
# e.g. Z6^4 is G = [[1,-1,0,0,0,0],[0,1,-1,0,0,0],[0,0,1,-1,0,0],[0,0,0,1,-1,0]]

#G = [(1, -1, 0, 0, 0, 0), (0, 1, -1, 0, 0, 0), (0, 0, 1, -1, 0, 0), (0, 0, 0, 0, 2, -2)] #2by2 case with 3 h22 forms
G=[[1,-1,0,0,0,0],[0,3,-3,0,0,0],[0,0,3,-3,0,0],[0,0,0,0,3,-3]] #81 by 81 case
def h22_even(G,h22_betas):
    '''
    Find the h22 forms even under the above group action G
    even_h22_betas: result, the even h22 forms under the action G
    '''
    even_h22_betas = []
    for beta in h22_betas:
        if all([vector(beta)*vector(g)%6 == 0 for g in G]):
            even_h22_betas.append(beta)
    return even_h22_betas        
h22_even_len=len(h22_even(G,h22_beta))
def h31_even(G,h31_betas):
    '''
    Find the h31 forms even under the above group action G
    even_h31_betas: result, the even h31 forms under the action G
    '''
    even_h31_betas = []
    for beta in h31_betas:
        if all([vector(beta)*vector(g)%6 == 0 for g in G]):
            even_h31_betas.append(beta)
    return even_h31_betas        

In [13]:
#generating the list of 3 decomposable forms, such that we may identify
#the n-decomposability of our leftover symmetric forms
def list_of_3decomposable():
    '''
    List of 3 decomposable forms, under the convention of the paper. TODO : list all of them regardless of convention
    target: result, the list of 3 decomposable forms respecting the convention
    temp: stores the possible results and generates permutations
    '''
    import numpy as np
    from itertools import permutations
    target=[]
    temp=[]
    for a in range(5):
        for b in range(5):
            for c in range(5):
                target.append([a,4-a,b,4-b,c,4-c])
    for i in range(len(target)):
        temp.append(list(permutations(target[i])))
    target=temp        
    return target
list3d=list_of_3decomposable()
#same as above for 1-decomposable
def list_of_1decomposable():
    import numpy as np
    from itertools import permutations
    '''
    Same as above but for 1 decomposable. The last case is indecomposable and automatically falls into that category if not 
    3 decomposable or 1 decomposable. TODO : list all of them regardless of convention
    target: result, the list of 1 decomposable forms respecting the convention of the paper
    temp: stores the possible results and generates permutations
    '''
    target=[]
    temp=[]
    for a in range(5):
        target.append([a,4-a,0,2,3,3])
    for i in range(len(target)):
        temp.append(list(permutations(target[i])))
    target=temp
    return target
list1d=list_of_1decomposable()
#Note that in a possible scale up, this operation is a bit redundant as we need to specify the permutation group
#we work with anyway and it becomes simple to check if a given residue is n-decomposable

In [14]:
#Here we compute the gammas appearing in the scalar product
def compute_Gamma(h22):
    '''
    Computes the product of gammas functions appearing in the scalar product
    result: result, stores the product of gammas functions
    prod: product of gamma functions, symbolic variable
    '''
    result=[]
    prod=1
    for i in range(len(h22)):
        for j in range(len(h22[i])):
            prod=prod*gamma((h22[i][j]+1)/6)
        result.append(prod.full_simplify())
        prod=1
    return result
gammas=compute_Gamma(nocc(h22_even(G,h22_beta)))
fullgammas=compute_Gamma((h22_even(G,h22_beta)))
print(gammas) #only a single product of gammas in a pair of complex conjugates
#note that to scale up, we can simply use euler's reflection formula to proceed in linear time
#this is good enough for now

[gamma(5/6)^3*gamma(1/6)^3, pi*gamma(5/6)^2*gamma(1/6)^2, pi*gamma(5/6)^2*gamma(1/6)^2, sqrt(pi)*gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2, pi*gamma(5/6)^2*gamma(1/6)^2, gamma(5/6)^3*gamma(1/6)^3, pi*gamma(5/6)^2*gamma(1/6)^2, sqrt(pi)*gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2, pi*gamma(5/6)^2*gamma(1/6)^2, gamma(5/6)^3*gamma(1/6)^3, gamma(5/6)^2*gamma(2/3)*gamma(1/3)*gamma(1/6)^2, pi*gamma(5/6)^2*gamma(1/6)^2, gamma(5/6)^2*gamma(2/3)*gamma(1/3)*gamma(1/6)^2, gamma(5/6)^3*gamma(1/6)^3, gamma(5/6)^2*gamma(1/3)^4, sqrt(pi)*gamma(5/6)*gamma(2/3)*gamma(1/3)^3, gamma(2/3)^3*gamma(1/3)^3, sqrt(pi)*gamma(5/6)*gamma(2/3)*gamma(1/3)^3, sqrt(pi)*gamma(5/6)*gamma(2/3)*gamma(1/3)^3, gamma(2/3)^3*gamma(1/3)^3, sqrt(pi)*gamma(5/6)*gamma(2/3)*gamma(1/3)^3, gamma(5/6)*gamma(2/3)^2*gamma(1/3)^2*gamma(1/6), gamma(2/3)^3*gamma(1/3)^3, pi*gamma(2/3)^2*gamma(1/3)^2, gamma(2/3)^3*gamma(1/3)^3, gamma(5/6)*gamma(2/3)^2*gamma(1/3)^2*gamma(1/6), pi*gamma(5/6)^2*gamma(1/6)^2, pi^2*gamma(5/6)*gamma(1/6), pi^(3/2)*gamma(2/3

In [15]:
#the function that does not depend on beta' and can therefore be introduced in the scalar product
def compute_xi(arr):
    '''
    Compute the xi term
    result: stores the results
    prod: stores the product of the xi terms.
    '''
    result=[]
    prod=1
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            prod=prod*(e^(2*I*pi/6*(arr[i][j]+1))-1)
        result.append(prod.full_simplify())
        prod=1
    return result
xi=compute_xi(nocc(h22_even(G,h22_beta)))
fullxi=compute_xi((h22_even(G,h22_beta)))
print(xi) #only a single xi for every pair of complex conjugate
#similarly with the gammas, there are not that many xi to compute in general so this is good enough
#although not optimal ( permutations induce some redundancy )

[1, 4, 4, 6, 4, 1, 4, 6, 4, 1, 3, 4, 3, 1, 9, 18, 27, 18, 18, 27, 18, 9, 27, 36, 27, 9, 4, 16, 24, 16, 4, 12, 16, 12, 4, 16, 24, 16, 16, 48, 64]


In [16]:
#now checkig if the h22 form we have is indeed 3-decomposable
def is_3decomposable(h22):
    '''
    Checks if the h22 forms h22 is 3-decomposable, returns a boolean
    IMPORTANT : only 3 decomposable wrt the convention picked
    '''
    import numpy as np
    h22=np.array(h22)
    for i in range(len(list3d)):
        for j in range(len(list3d[i])):
            if np.array_equal(h22,list3d[i][j]):
                return True
    return False

#same as before
def is_1decomposable(h22):
    '''
    Checks if the h22 forms h22 is 1-decomposable, returns a boolean
    IMPORTANT : only 1 decomposable wrt the convention picked
    '''
    import numpy as np
    h22=np.array(h22)
    for i in range(len(list1d)):
        for j in range(len(list1d[i])):
            if np.array_equal(h22,list1d[i][j]):
                return True
    return False

In [17]:
#The constant of proportionalities stemming from complex conjugation 
def compute_proportionality(arr):
    '''
    Compute the constant of proportionality with respect to the convention picked.
    '''
    result=[]
    counter=0
    for i in range(len(arr)):
        if is_3decomposable(arr[i])==True:
            result.append(-1)
        elif is_1decomposable(arr[i])==True:
            result.append(-2^(1/3))
        else:
            result.append(-2^(-2/3))
    return result

In [18]:
#just to check !
h22=nocc(h22_even(G,h22_beta))
h22_with_cc=h22_even(G,h22_beta)
prop=compute_proportionality(h22)
print(prop) #one constant for each pair

[-1, -1, -1, -2^(1/3), -1, -1, -1, -2^(1/3), -1, -1, -1, -1, -1, -1, -1/2*2^(1/3), -1/2*2^(1/3), -1, -1/2*2^(1/3), -1/2*2^(1/3), -1, -1/2*2^(1/3), -1, -1, -1, -1, -1, -1, -1, -2^(1/3), -1, -1, -1, -1, -1, -1, -1, -2^(1/3), -1, -1, -1, -1]


In [19]:
print(h22)
        

[[0, 0, 0, 4, 4, 4], [0, 0, 2, 2, 4, 4], [0, 0, 2, 4, 2, 4], [0, 0, 2, 4, 3, 3], [0, 0, 2, 4, 4, 2], [0, 0, 4, 0, 4, 4], [0, 0, 4, 2, 2, 4], [0, 0, 4, 2, 3, 3], [0, 0, 4, 2, 4, 2], [0, 0, 4, 4, 0, 4], [0, 0, 4, 4, 1, 3], [0, 0, 4, 4, 2, 2], [0, 0, 4, 4, 3, 1], [0, 0, 4, 4, 4, 0], [1, 1, 1, 1, 4, 4], [1, 1, 1, 3, 2, 4], [1, 1, 1, 3, 3, 3], [1, 1, 1, 3, 4, 2], [1, 1, 3, 1, 2, 4], [1, 1, 3, 1, 3, 3], [1, 1, 3, 1, 4, 2], [1, 1, 3, 3, 0, 4], [1, 1, 3, 3, 1, 3], [1, 1, 3, 3, 2, 2], [1, 1, 3, 3, 3, 1], [1, 1, 3, 3, 4, 0], [2, 2, 0, 0, 4, 4], [2, 2, 0, 2, 2, 4], [2, 2, 0, 2, 3, 3], [2, 2, 0, 2, 4, 2], [2, 2, 0, 4, 0, 4], [2, 2, 0, 4, 1, 3], [2, 2, 0, 4, 2, 2], [2, 2, 0, 4, 3, 1], [2, 2, 0, 4, 4, 0], [2, 2, 2, 0, 2, 4], [2, 2, 2, 0, 3, 3], [2, 2, 2, 0, 4, 2], [2, 2, 2, 2, 0, 4], [2, 2, 2, 2, 1, 3], [2, 2, 2, 2, 2, 2]]


In [90]:
'''

IMPORTANT : the 81 by 81 case does NOT respect the convention used in the paper hence manually entering the constants of proportionality
to match the gammas and xi.

'''

prop=[-1,-1,-1,-2^(1/3),-1,-1,-1,-2^(1/3),-1,-1,-1,-1,-1,-1,-1/2*2^(1/3),-2^(-1/3),-1,-2^(-1/3),-2^(-1/3),-1,-2^(-1/3),
     -1,-1,-1,-1,-1,-1,-1,-2^(1/3),-1,-1,-1,-1,-1,-1,-1,-2^(1/3),-1,-1,-1,-1]

In [92]:
#compute the part of the rescaling that does not belong in the lattice
def compute_alpha(gamma1,xi1):
    '''
    Finally computes the overall constant, depends on gamma, xi and prop
    -result: result only for h22 forms
    -ccresult: result for their complex conjugate according to the convention
    '''
    result=[]
    resultcc=[]
    for i in range(len(gamma1)):
        if i!=len(gamma1)-1:
            result.append(1/xi[i]*1/gamma1[i]*(6^5)*4*pi*I)
            resultcc.append((1/xi[i]*1/gamma1[i]*(6^5)*4*pi*I)*prop[i])
        else:
            result.append(1/xi[i]*1/gamma1[i]*(6^5)*4*pi*I)
            resultcc.append((1/xi[i]*1/gamma1[i]*(6^5)*4*pi*I)*prop[i])
    return result,resultcc
alphas,ccalphas=compute_alpha(gammas,xi)
import numpy as np
alphas=np.array(alphas)
ccalphas=np.array(ccalphas) #the constant terms for a residue ( alphas ) and for its complex
#conjugate (cc alphas)

In [93]:
print(alphas)
print(ccalphas)

[31104*I*pi/(gamma(5/6)^3*gamma(1/6)^3) 7776*I/(gamma(5/6)^2*gamma(1/6)^2)
 7776*I/(gamma(5/6)^2*gamma(1/6)^2)
 5184*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2)
 7776*I/(gamma(5/6)^2*gamma(1/6)^2) 31104*I*pi/(gamma(5/6)^3*gamma(1/6)^3)
 7776*I/(gamma(5/6)^2*gamma(1/6)^2)
 5184*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2)
 7776*I/(gamma(5/6)^2*gamma(1/6)^2) 31104*I*pi/(gamma(5/6)^3*gamma(1/6)^3)
 10368*I*pi/(gamma(5/6)^2*gamma(2/3)*gamma(1/3)*gamma(1/6)^2)
 7776*I/(gamma(5/6)^2*gamma(1/6)^2)
 10368*I*pi/(gamma(5/6)^2*gamma(2/3)*gamma(1/3)*gamma(1/6)^2)
 31104*I*pi/(gamma(5/6)^3*gamma(1/6)^3)
 3456*I*pi/(gamma(5/6)^2*gamma(1/3)^4)
 1728*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3)
 1152*I*pi/(gamma(2/3)^3*gamma(1/3)^3)
 1728*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3)
 1728*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3)
 1152*I*pi/(gamma(2/3)^3*gamma(1/3)^3)
 1728*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3)
 3456*I*pi/(gamma(5/6)*gamma(2/3)^2*gamma(1/3)^2*gamma(1/6

In [78]:
#computing the invariant residues form and making them global
#print checks

print(nocc(h22_even(G,h22_beta))) #pick one representant from each h22 pair
nocc(h31_even(G,h31_beta))
h22size=len(nocc(h22_even(G,h22_beta)))
print(h22size) #total number of h22 forms without complex conjugates
h31size=len(nocc(h31_even(G,h31_beta)))
allh31=h31_even(G,h31_beta)
h22=nocc(h22_even(G,h22_beta))
allh22=h22_even(G,h22_beta)
print(h22_even(G,h22_beta)) # all h22 forms
print(len(h22_even(G,h22_beta))) #total number of h22 forms with complex conjugates
h31=nocc(h31_even(G,h31_beta))
print(h31_even(G,h31_beta)) # all h31 forms
print(len(nocc(h31_even(G,h31_beta)))) # number of h31 forms without complex conjugates

[[0, 0, 0, 4, 4, 4], [0, 0, 2, 2, 4, 4], [0, 0, 2, 4, 2, 4], [0, 0, 2, 4, 3, 3], [0, 0, 2, 4, 4, 2], [0, 0, 4, 0, 4, 4], [0, 0, 4, 2, 2, 4], [0, 0, 4, 2, 3, 3], [0, 0, 4, 2, 4, 2], [0, 0, 4, 4, 0, 4], [0, 0, 4, 4, 1, 3], [0, 0, 4, 4, 2, 2], [0, 0, 4, 4, 3, 1], [0, 0, 4, 4, 4, 0], [1, 1, 1, 1, 4, 4], [1, 1, 1, 3, 2, 4], [1, 1, 1, 3, 3, 3], [1, 1, 1, 3, 4, 2], [1, 1, 3, 1, 2, 4], [1, 1, 3, 1, 3, 3], [1, 1, 3, 1, 4, 2], [1, 1, 3, 3, 0, 4], [1, 1, 3, 3, 1, 3], [1, 1, 3, 3, 2, 2], [1, 1, 3, 3, 3, 1], [1, 1, 3, 3, 4, 0], [2, 2, 0, 0, 4, 4], [2, 2, 0, 2, 2, 4], [2, 2, 0, 2, 3, 3], [2, 2, 0, 2, 4, 2], [2, 2, 0, 4, 0, 4], [2, 2, 0, 4, 1, 3], [2, 2, 0, 4, 2, 2], [2, 2, 0, 4, 3, 1], [2, 2, 0, 4, 4, 0], [2, 2, 2, 0, 2, 4], [2, 2, 2, 0, 3, 3], [2, 2, 2, 0, 4, 2], [2, 2, 2, 2, 0, 4], [2, 2, 2, 2, 1, 3], [2, 2, 2, 2, 2, 2]]
41
[[0, 0, 0, 4, 4, 4], [0, 0, 2, 2, 4, 4], [0, 0, 2, 4, 2, 4], [0, 0, 2, 4, 3, 3], [0, 0, 2, 4, 4, 2], [0, 0, 4, 0, 4, 4], [0, 0, 4, 2, 2, 4], [0, 0, 4, 2, 3, 3], [0, 0, 4, 2, 4,

In [34]:
#generating the lattice and print-checking
#note that to scale this up, it may (?) prove necessary to rewrite the .gens() and .dual_lattice() methods properly 

generator=L_cond(nocc(h22_even(G,h22_beta))).gens() #uses row echelon form extremely slow
print(generator) #generators of the lattice of Z(beta,beta') appearing for h22
lengen=len(generator[0]) #numbers of generators
SP = matrix.identity(3*len(nocc(h22_even(G,h22_beta)))) #euclidian scalar product to convert
Lattice=IntegralLattice(SP, generator)
DualLattice=Lattice.dual_lattice() #using sage's built-in function to automatically compute the dual
DualLatticeGens=DualLattice.gens()
matrixdualgens=matrix(DualLatticeGens)
print(DualLatticeGens) #prints out the generators of the dual lattice of Z(beta,beta')


((1, 0, -1, 0, 2, -2, 0, 2, -2, 0, 0, 0, 0, 2, -2, 2, 1, -3, 2, 1, -3, 0, 0, 0, 2, 1, -3, 2, 1, -3, 0, 0, 0, 2, 1, -3, 0, 3, -3, 2, 4, -6, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 1, 2, -3, 0, 0, 0, 0, 0, 0, 1, 2, -3, 3, 0, -3, 0, 10, -10, 3, 0, -3, 0, 2, -2, 0, 2, -2, 0, 0, 0, 0, 5, -5, 0, 2, -2, 0, 0, 0, 0, 5, -5, 0, 0, 0, 0, 2, -2, 2, 7, -9, 0, 0, 0, 2, 1, -3, 2, 7, -9, 3, 0, -3, 5, -5, 0), (0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 0, 0, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 0, 0, 0, 1, -1, 0, 1, -1, 0, 0, 0, 0, 1, -1, 0, 3, -3, 0, 4, -4, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 1, 0, -1, 3, 0, -3, 0, 8, -8, 3, 0, -3, 1, 2, -3, 1, 2, -3, 0, 0, 0, 1, 5, -6, 1, 2, -3, 0, 0, 0, 1, 5, -6, 3, 0, -3, 4, 14, -18, 1, 2, -3, 0, 0, 0, 1, 5, -6, 1, 2, -3, 0, 3, -3, 7, -7, 0), (0, 0, 0, 1, 1, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, -4, 1, 1, -2, 0, 0, 0, 1, 1, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

((1/18, 0, -1/18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/9, -1/9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/9, 1/18, -1/6, 0, 0, 0, 0, 0, 0, 0, 1/6, -1/6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/6, 0, -1/6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/9, -1/9, 0, 0, 0, 0, 0, 0, 0, 1/9, -1/9, 0, 0, 0, 0, 2/9, -2/9, 0, 1/9, -1/9, 0, 0, 0, 0, 2/9, -2/9, 1/18, 0, -1/18, 0, 1/2, -1/2, 1/9, -1/9, 0), (0, 1/18, -1/18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2/9, -2/9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/18, -1/18, 0, 0, 0, 0, 0, 0, 1/6, 0, -1/6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/6, 1/3, -1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2/9, -2/9, 0, 0, 0, 0, 0, 0, 0, 2/9, -2/9, 0, 0, 0, 2/9, 0, -2/9, 0, 2/9, -2/9, 0, 0, 0, 2/9, 0, -2/9, 1/18, 1/9, -1/6, 0, 1/2, -1/2, 1/18, -1/18, 0), (0, 0, 0, 1/9, 0, -1/9, 0, 0, 0, 0, 0, 0, 0, 1/9, -1/9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/9, -1/9, 0, 0, 0, 0, 

In [22]:
#notice how the generators are ordered : the maximally symmetric form is always at the end :
#the last triplet will of each generator will be a Q-linear combination of the generator of A1*
#while the rest are in general in A2*

In [35]:
#the base lattice, which we know our lattice is a sublattice of, expressed in Q^3.
def A2():
    '''
    Generates the basis of A2* matching the convention of sagemath implementation in QQ^3
    returns LZZ, all the A2* lattice and the generators only respectively
    '''
    adual=[2/3,-1/3,-1/3]
    cdual=[1/3,1/3,-2/3]
    resultdual=[adual,[1,0,-1],cdual,[-2/3,1/3,1/3],[-1,0,1],[-1/3,-1/3,2/3]]
    resultdualgens=[adual,[1,0,-1],cdual] #Note the following : we add a third list when we only need 2 in principle 
    #( since the A2 lattice is 2 dimensional ). This is because we will need to invert the system and it can be
    #performed in sage using built-in functions ( which are much faster ) only if the matrix to invert is 
    #square, thus it is necessary to add a third vector so the dimensions of the source and the target match
    return LZZ, resultdual,resultdualgens
A2lattice,A2starlattice,A2starlatticegens=A2()
print(A2lattice)
print(A2starlattice) #vectors of A2*

[[1, -1, 0], [1, 0, -1], [0, 1, -1], [-1, 1, 0], [-1, 0, 1], [0, -1, 1]]
[[2/3, -1/3, -1/3], [1, 0, -1], [1/3, 1/3, -2/3], [-2/3, 1/3, 1/3], [-1, 0, 1], [-1/3, -1/3, 2/3]]


In [36]:
#Same before, but now it's in a complex basis
def A2star_in_root_basis():
    '''
    Generates the basis of A2* in CC^3 this time, returns all the A2* lattice and generators respectively
    '''
    c=e^(5*I*pi/6)/sqrt(3)
    a=e^(2*I*pi/12)/sqrt(3)
    result=[a,a+c,c,-a,-a-c,-c]
    rootsgens=[a,a+c,c]
    return result,rootsgens

rootbasisdual,rootsdualgens=A2star_in_root_basis()
print(rootbasisdual)
print(rootsdualgens)#again using the trick of adding a (redundant) vector so the dimensions match

[1/6*sqrt(3)*(sqrt(3) + I), 1/6*sqrt(3)*(sqrt(3) + I) - 1/6*sqrt(3)*(sqrt(3) - I), -1/6*sqrt(3)*(sqrt(3) - I), -1/6*sqrt(3)*(sqrt(3) + I), -1/6*sqrt(3)*(sqrt(3) + I) + 1/6*sqrt(3)*(sqrt(3) - I), 1/6*sqrt(3)*(sqrt(3) - I)]
[1/6*sqrt(3)*(sqrt(3) + I), 1/6*sqrt(3)*(sqrt(3) + I) - 1/6*sqrt(3)*(sqrt(3) - I), -1/6*sqrt(3)*(sqrt(3) - I)]


In [37]:
#Well we have to deal with A1 as well, so we proceed as the previous case.
#In the "complex" base with bar(a)*b+a*bar(b) inner form, the generator is 1, while in Q^3 it is [1,-1,0] 
def A1():
    '''
    A1 to deal with the self-conjugate h22 form, return the generators in CC^3 and QQ^3 respectively
    '''
    return [1],[1,-1,0]

In [38]:
#print-check and declaring a global variable 
gensA1roots,gensA1Z=A1()
print(gensA1Z) #generators of A1

#ended up not using any of those functions so need to get rid of them !

[1, -1, 0]


In [39]:
#each vector in the generators is currently written as members of Q^n, but we are really interested in triplets formed by
#the coordinates of each vectors, as their linear combinations in terms of generators of A2* will give us the basis of the 
#lattice in terms of complex numbers
def split_gens(gens):
    '''
    Splits the generators in list of 3 to match the embedding in QQ^3. TODO : implement it with numpy, faster
    returns the generators of the dual lattice splitted in lists of length 3
    
    Format : [list of all generators [ith generators [split 3 by 3]]]
    '''
    temp2=[]
    dualsplit=[]
    for i in range(len(gens)):
        for j in range(0,len(gens[i]),3):
            temp2.append([gens[i][j],gens[i][j+1],gens[i][j+2]])    
        dualsplit.append(temp2)
        temp2=[]
    return dualsplit

dualsplit=split_gens(DualLatticeGens)
print(dualsplit) #Splitting the generators in terms of copies of Z3



[[[1/18, 0, -1/18], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1/9, -1/9], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [1/9, 1/18, -1/6], [0, 0, 0], [0, 0, 0], [0, 1/6, -1/6], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [1/6, 0, -1/6], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1/9, -1/9], [0, 0, 0], [0, 0, 0], [0, 1/9, -1/9], [0, 0, 0], [0, 2/9, -2/9], [0, 1/9, -1/9], [0, 0, 0], [0, 2/9, -2/9], [1/18, 0, -1/18], [0, 1/2, -1/2], [1/9, -1/9, 0]], [[0, 1/18, -1/18], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 2/9, -2/9], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1/18, -1/18], [0, 0, 0], [0, 0, 0], [1/6, 0, -1/6], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [1/6, 1/3, -1/2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 2/9, -2/9], [0, 0, 0], [0, 0, 0], [0, 2/9, -2/9], [0, 0, 0], [2/9, 0, -2/9], [0, 2/9, -2/9], [0, 0, 0], [2/9, 0, -2/9],

In [40]:
#Note : this is not the best way to proceed and I would fuse the previous and the next function together in
#a scaled up version and proceed in parallel. Also changing the shape so much is costly in time
#but it is readable and I could track mistakes this way

In [41]:
#We need to get the linear combination for each triplet of entries of each generator of the lattice
def get_linear_combination(splitted):
    '''
    Find the linear combination of the dual lattice in terms of A2* lattice embedded in Q3 TODO: implement it using numpy
    -target: flattening of the splitted generators
    -augmented: list of 3 generators of the lattice to perform gaussian elimination
    -coeff: result, coefficients of linear combination for each vector of the basis
    '''
    target=[]
    for i in range(len(splitted)):
        for j in range(len(splitted[i])):
            target.append(splitted[i][j])
    augmented=[[2/3, -1/3, -1/3], [1, 0, -1], [1/3, 1/3, -2/3]] #using the trick as explained before : add 1 vector to make
    #the matrix square
    A1gen=[1,-1,0]
    coeff=[] 
    for i in range(len(target)):
        #gaussian elimination to get the coefficients that lie in A2*
        if (i+1)%(h22size)!=0:
            augmented.append(target[i])
            augmented=list(map(list, zip(*augmented)))
            augmented=matrix(augmented)
            augmented=augmented.rref()
            augmented=list(map(list, zip(*augmented)))
            coeff.append(augmented[3])
            augmented=[[2/3, -1/3, -1/3], [1, 0, -1], [1/3, 1/3, -2/3]]
        else: #else do it for the A1 part.
            coeff.append([target[i][0]/A1gen[0],0,0])
    return coeff
solutions=get_linear_combination(dualsplit)
print(solutions) #getting the linear combinations of our splitted generators in terms of generators
#of A2* using simple row reduction, hence adding a third vector being useful to automate the process.

[[0, 1/18, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-1/3, 2/9, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-1/6, 2/9, 0], [0, 0, 0], [0, 0, 0], [-1/2, 1/3, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1/6, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-1/3, 2/9, 0], [0, 0, 0], [0, 0, 0], [-1/3, 2/9, 0], [0, 0, 0], [-2/3, 4/9, 0], [-1/3, 2/9, 0], [0, 0, 0], [-2/3, 4/9, 0], [0, 1/18, 0], [-3/2, 1, 0], [1/9, 0, 0], [-1/6, 1/9, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-2/3, 4/9, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-1/6, 1/9, 0], [0, 0, 0], [0, 0, 0], [0, 1/6, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-1, 5/6, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [-2/3, 4/9, 0], [0, 0, 0], [0, 0, 0], [-2/3, 4/9, 0], [0, 0, 0], [0, 2/9, 0], [-2/3, 4/9, 0], [0, 0, 0], [0, 2/9, 0], [-1/3, 5/18, 0], [-3/2, 1, 0], [1/18, 

In [42]:
#Now we have the coefficients of the linear combination so we can just generate the lattice in terms of complex numbers
def recover_dual(sols):
    '''
    Converts the solutions from QQ^3 to CC^3
    result: result, the generators of the lattice in CC^3 in terms of linear combination of generators of A2* in CC^3
    '''
    result=[]
    for i in range(len(sols)):
        for j in range(len(sols[i])):
            if (i+1)%(h22size)!=0:
                result.append(sols[i][j]*rootsdualgens[j])
            else:
                result.append(sols[i][j])
    return result

In [43]:
dualsum=recover_dual(solutions) #converts to complex generators but we need to rearrange the format
#of the output

In [44]:
print(dualsum) #just checking !
    

[0, 1/108*sqrt(3)*(sqrt(3) + I) - 1/108*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/18*sqrt(3)*(sqrt(3) + I), 1/27*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/36*sqrt(3)*(sqrt(3) + I), 1/27*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, -1/12*sqrt(3)*(sqrt(3) + I), 1/18*sqrt(3)*(sqrt(3) + I) - 1/18*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/36*sqrt(3)*(sqrt(3) + I) - 1/36*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/18*sqrt(3)*(sqrt(3) + I), 1/27*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, -1/18*sqrt(3)*(sqrt(3) + I), 1/27*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, -1/9*sqrt(3)*(sqrt(3) + I), 2/27*sqrt(3)*(sqrt(3) + I) - 2/27*sqrt(3)*(sqrt(3) - I), 0, -1/18*sqrt(3)*(sqrt(3) + I), 1/27*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 

In [45]:
#Since the output was just a list, we need to put it in matrix form
#Would have done it with numpy if it was on python but I do not know the precise 
#inner workings of sagemath wrt numpy
def arrange_array(summed):
    '''
    TODO rewrite this using np.reshape()
    Previous result is a 1D array, convert it to the appropriate format by summing the terms 3 by 3 until the appropriate length is met
    temp and temp2: temporary array, stores the sums of 3 terms and puts things in the right format respectively
    summs: stores the summs of 3 terms
    result: result, in the right format
    '''
    temp=[]
    result=[]
    summ=0
    for i in range(0,len(summed),3):
        for j in range(3):
            summ+=summed[i+j]
        temp.append(summ)
        summ=0
    temp=np.array(temp)
    
    temp2=[]
    for i in range(0,len(temp),h22size):
        for j in range(h22size):
            temp2.append(temp[i+j])
        temp2=np.array(temp2)
        result.append(list(temp2.flatten()))
        temp2=[]
    
    return result
#the output is actually the transpose of what we want 

In [46]:
lattice_in_roots=arrange_array(dualsum) 

In [47]:
print(lattice_in_roots) #checking

[[1/108*sqrt(3)*(sqrt(3) + I) - 1/108*sqrt(3)*(sqrt(3) - I), 0, 0, 0, -1/54*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 0, 1/108*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, -1/36*sqrt(3)*(sqrt(3) + I) - 1/18*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 1/36*sqrt(3)*(sqrt(3) + I) - 1/36*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, -1/54*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, 0, -1/54*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, -1/27*sqrt(3)*(sqrt(3) + I) - 2/27*sqrt(3)*(sqrt(3) - I), -1/54*sqrt(3)*(sqrt(3) + I) - 1/27*sqrt(3)*(sqrt(3) - I), 0, -1/27*sqrt(3)*(sqrt(3) + I) - 2/27*sqrt(3)*(sqrt(3) - I), 1/108*sqrt(3)*(sqrt(3) + I) - 1/108*sqrt(3)*(sqrt(3) - I), -1/12*sqrt(3)*(sqrt(3) + I) - 1/6*sqrt(3)*(sqrt(3) - I), 1/9], [-1/108*sqrt(3)*(sqrt(3) + I) - 1/54*sqrt(3)*(sqrt(3) - I), 0, 0, 0, -1/27*sqrt(3)*(sqrt(3) + I) - 2/27*sqrt(3)*(sqrt(3) - I), 0, 0, 0, 0, 0, 0, 0, 0, -1/108*sqrt(3)*(sqrt(3) + I) - 1/54*sqrt(3)*(sqrt(3) - I), 0,

In [79]:
#now putting the basis in the right format for the final computation
#again very dirty but I was tired of compiling every time
change_basis=matrix(lattice_in_roots).transpose()
print(change_basis)
print(change_basis.nrows())
print(change_basis.ncols())

[ 1/108*sqrt(3)*(sqrt(3) + I) - 1/108*sqrt(3)*(sqrt(3) - I)  -1/108*sqrt(3)*(sqrt(3) + I) - 1/54*sqrt(3)*(sqrt(3) - I)                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                          0                                                        

In [94]:
n= vector(list(var('x%d' % i) for i in range(1,len(allh22)+1))) #generates the symbols for each h22 forms

In [95]:
symbolicvector=change_basis*n
print(len(symbolicvector))

41


In [96]:
#generates the symbols with the appropriate coefficients in front for all h22 forms

ccsymbolicvector=[]
for i in range(len(symbolicvector)):
    ccsymbolicvector.append(ccalphas[i]*(symbolicvector[i].conjugate()))
    if i!=40:
        symbolicvector[i]=alphas[i]*symbolicvector[i]*2*(2*pi)^4/(2^7*3^5)
    else:
        symbolicvector[i]=alphas[i]*symbolicvector[i]*4*(2*pi)^4/(2^7*3^5)
ccsymbolicvector=vector(ccsymbolicvector)

In [97]:
#returns the quadratic form associated with the above generators

for i in range(len(symbolicvector)):
    print((symbolicvector[i]*ccsymbolicvector[i]).full_simplify())
    print('+')

1024*(pi^6*x1^2 + pi^6*x1*x2 + 7*pi^6*x2^2)/(gamma(5/6)^6*gamma(1/6)^6)
+
256*(pi^4*x3^2 + pi^4*x3*x4 + 7*pi^4*x4^2)/(gamma(5/6)^4*gamma(1/6)^4)
+
256*(pi^4*x5^2 + pi^4*x5*x6 + 7*pi^4*x6^2)/(gamma(5/6)^4*gamma(1/6)^4)
+
1024*(2^(1/3)*pi^5*x7^2 + 5*2^(1/3)*pi^5*x7*x8 + 7*2^(1/3)*pi^5*x8^2)/(gamma(5/6)^2*gamma(2/3)^4*gamma(1/6)^4)
+
256*(7*pi^4*x1^2 + 42*pi^4*x1*x10 + 63*pi^4*x10^2 + 28*pi^4*x2^2 + 7*pi^4*x3^2 + 28*pi^4*x4^2 + 7*pi^4*x5^2 + 28*pi^4*x6^2 + 9*pi^4*x9^2 + 28*(pi^4*x1 + 3*pi^4*x10)*x2 + 14*(pi^4*x1 + 3*pi^4*x10 + 2*pi^4*x2)*x3 + 28*(pi^4*x1 + 3*pi^4*x10 + 2*pi^4*x2 + pi^4*x3)*x4 + 14*(pi^4*x1 + 3*pi^4*x10 + 2*pi^4*x2 + pi^4*x3 + 2*pi^4*x4)*x5 + 28*(pi^4*x1 + 3*pi^4*x10 + 2*pi^4*x2 + pi^4*x3 + 2*pi^4*x4 + pi^4*x5)*x6 + 15*(pi^4*x1 + 3*pi^4*x10 + 2*pi^4*x2 + pi^4*x3 + 2*pi^4*x4 + pi^4*x5 + 2*pi^4*x6)*x9)/(gamma(5/6)^4*gamma(1/6)^4)
+
1024*(pi^6*x11^2 + pi^6*x11*x12 + 7*pi^6*x12^2)/(gamma(5/6)^6*gamma(1/6)^6)
+
256*(pi^4*x13^2 + pi^4*x13*x14 + 7*pi^4*x14^2)/(gamma(5/6)^4*gamma(

256/81*(4*pi^6*x1^2 + 8*pi^6*x1*x11 + 4*pi^6*x11^2 + 124*pi^6*x12^2 + 252*pi^6*x13^2 + 36*pi^6*x17^2 + 28*pi^6*x19^2 + 124*pi^6*x2^2 + 112*pi^6*x20^2 + 9*pi^6*x21^2 + 9*pi^6*x22^2 + 252*pi^6*x27^2 + 468*pi^6*x28^2 + 252*pi^6*x3^2 + 112*pi^6*x33^2 + 448*pi^6*x34^2 + 112*pi^6*x39^2 + 448*pi^6*x40^2 + 36*pi^6*x43^2 + 252*pi^6*x44^2 + 448*pi^6*x45^2 + 112*pi^6*x46^2 + 144*pi^6*x49^2 + 252*pi^6*x5^2 + 1008*pi^6*x50^2 + 36*pi^6*x9^2 + 16*(pi^6*x1 + pi^6*x11)*x12 + 12*(pi^6*x1 + pi^6*x11 + 29*pi^6*x12)*x13 + 12*(pi^6*x1 + pi^6*x11 + 11*pi^6*x12 + 15*pi^6*x13)*x17 + 4*(pi^6*x1 + pi^6*x11 + 29*pi^6*x12 + 42*pi^6*x13 + 15*pi^6*x17)*x19 + 4*(4*pi^6*x1 + 4*pi^6*x11 + 62*pi^6*x12 + 87*pi^6*x13 + 33*pi^6*x17 + 29*pi^6*x19)*x2 + 8*(pi^6*x1 + pi^6*x11 + 29*pi^6*x12 + 42*pi^6*x13 + 15*pi^6*x17 + 14*pi^6*x19 + 29*pi^6*x2)*x20 + 6*(pi^6*x1 + pi^6*x11 + 11*pi^6*x12 + 15*pi^6*x13 + 6*pi^6*x17 + 5*pi^6*x19 + 11*pi^6*x2 + 10*pi^6*x20)*x21 + 3*(4*pi^6*x1 + 4*pi^6*x11 + 8*pi^6*x12 + 6*pi^6*x13 + 6*pi^6*x17 + 2

16*(7*pi^2*x1^2 + 28*pi^2*x1*x2 + 28*pi^2*x2^2 + 7*pi^2*x3^2 + 28*pi^2*x4^2 + 28*pi^2*x53^2 + 7*pi^2*x54^2 + 7*pi^2*x55^2 + 28*pi^2*x56^2 + 9*pi^2*x59^2 + 63*pi^2*x60^2 + 14*(pi^2*x1 + 2*pi^2*x2)*x3 + 28*(pi^2*x1 + 2*pi^2*x2 + pi^2*x3)*x4 + 28*(pi^2*x1 + 2*pi^2*x2 + pi^2*x3 + 2*pi^2*x4)*x53 + 14*(pi^2*x1 + 2*pi^2*x2 + pi^2*x3 + 2*pi^2*x4 + 2*pi^2*x53)*x54 + 14*(pi^2*x1 + 2*pi^2*x2 + pi^2*x3 + 2*pi^2*x4 + 2*pi^2*x53 + pi^2*x54)*x55 + 28*(pi^2*x1 + 2*pi^2*x2 + pi^2*x3 + 2*pi^2*x4 + 2*pi^2*x53 + pi^2*x54 + pi^2*x55)*x56 + 15*(pi^2*x1 + 2*pi^2*x2 + pi^2*x3 + 2*pi^2*x4 + 2*pi^2*x53 + pi^2*x54 + pi^2*x55 + 2*pi^2*x56)*x59 + 3*(14*pi^2*x1 + 28*pi^2*x2 + 14*pi^2*x3 + 28*pi^2*x4 + 28*pi^2*x53 + 14*pi^2*x54 + 14*pi^2*x55 + 28*pi^2*x56 + 15*pi^2*x59)*x60)/(gamma(5/6)^2*gamma(1/6)^2)
+
64*(pi^4*x61^2 + pi^4*x61*x62 + 7*pi^4*x62^2)/(gamma(5/6)^4*gamma(1/6)^4)
+
64*(pi^4*x63^2 + 5*pi^4*x63*x64 + 7*pi^4*x64^2)/(gamma(5/6)^2*gamma(2/3)^2*gamma(1/3)^2*gamma(1/6)^2)
+
16*(7*pi^2*x1^2 + 28*pi^2*x1*x2 + 2

4*(pi^2*x1^2 + 3*pi^2*x1*x12 + 9*pi^2*x12^2 + 28*pi^2*x13^2 + 4*pi^2*x14^2 + 9*pi^2*x17^2 + 7*pi^2*x19^2 + 31*pi^2*x2^2 + 13*pi^2*x20^2 + 36*pi^2*x23^2 + 13*pi^2*x3^2 + 31*pi^2*x4^2 + 4*pi^2*x5^2 + 36*pi^2*x54^2 + 28*pi^2*x55^2 + 4*pi^2*x56^2 + 9*pi^2*x59^2 + pi^2*x6^2 + 28*pi^2*x61^2 + 31*pi^2*x62^2 + 9*pi^2*x65^2 + 9*pi^2*x71^2 + 9*pi^2*x77^2 + 63*pi^2*x78^2 + 36*pi^2*x9^2 + 2*(pi^2*x1 + 15*pi^2*x12)*x13 + 2*(2*pi^2*x1 + 3*pi^2*x12 + 2*pi^2*x13)*x14 + 3*(pi^2*x1 + 6*pi^2*x12 + 10*pi^2*x13 + 2*pi^2*x14)*x17 + (pi^2*x1 + 15*pi^2*x12 + 28*pi^2*x13 + 2*pi^2*x14 + 15*pi^2*x17)*x19 + (4*pi^2*x1 + 33*pi^2*x12 + 58*pi^2*x13 + 8*pi^2*x14 + 33*pi^2*x17 + 29*pi^2*x19)*x2 + (5*pi^2*x1 + 21*pi^2*x12 + 32*pi^2*x13 + 10*pi^2*x14 + 21*pi^2*x17 + 16*pi^2*x19 + 37*pi^2*x2)*x20 + 6*(pi^2*x1 + 6*pi^2*x12 + 10*pi^2*x13 + 2*pi^2*x14 + 6*pi^2*x17 + 5*pi^2*x19 + 11*pi^2*x2 + 7*pi^2*x20)*x23 + (5*pi^2*x1 + 21*pi^2*x12 + 32*pi^2*x13 + 10*pi^2*x14 + 21*pi^2*x17 + 16*pi^2*x19 + 37*pi^2*x2 + 26*pi^2*x20 + 42*pi^

6*x1^2 + 18*x1*x10 + 27/2*x10^2 + 3*(2*x1 + 3*x10)*x11 + 3/2*x11^2 + 15*(2*x1 + 3*x10 + x11)*x12 + 75/2*x12^2 + 21*(2*x1 + 3*x10 + x11 + 5*x12)*x13 + 147/2*x13^2 + 6*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13)*x14 + 6*x14^2 + 9*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14)*x18 + 27/2*x18^2 + 24*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18)*x19 + 96*x19^2 + 3*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18 + 8*x19)*x2 + 3/2*x2^2 + 12*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18 + 8*x19 + x2)*x20 + 24*x20^2 + 9*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18 + 8*x19 + x2 + 4*x20)*x22 + 27/2*x22^2 + 18*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18 + 8*x19 + x2 + 4*x20 + 3*x22)*x24 + 54*x24^2 + 9*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18 + 8*x19 + x2 + 4*x20 + 3*x22 + 6*x24)*x27 + 27/2*x27^2 + 18*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*x18 + 8*x19 + x2 + 4*x20 + 3*x22 + 6*x24 + 3*x27)*x28 + 54*x28^2 + 15*(2*x1 + 3*x10 + x11 + 5*x12 + 7*x13 + 2*x14 + 3*

In [ ]:
'''
TODO : below compute the ranks for 81 by 81 case
As this deals with symbolic variables and the number solutions for the representation of integers by the above quadratic
form is immense, this is too slow for now.
'''

In [151]:
coords=list((0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, -1, 0, 0, 0, -1, 0, 1, 0, 1, 2, -1, 0, 0, 0, 1, 1, 1, 2, 0, -1, -3))

In [155]:
alphascoords=[]
temp=[]
for j in range(len(coords)):
    if j>h22size:
        alphascoords.append(ccalphas[h22size-j-1]*coords[j])
    else:
        alphascoords.append(alphas[j-1]*coords[j])

print(alphascoords)

[0, 0, 0, 0, 5184*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2), 0, 0, 0, -5184*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2), 0, 0, 0, -7776*I/(gamma(5/6)^2*gamma(1/6)^2), 0, 0, 0, 1728*I*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1944*I/(pi*gamma(5/6)*gamma(1/6)), 0, 0, 0, 864*I/(gamma(2/3)^2*gamma(1/3)^2), 0, 3456*I*pi/(gamma(5/6)*gamma(2/3)^2*gamma(1/3)^2*gamma(1/6)), 0, 0, 0, 864*I*2^(2/3)*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3), 0, -864*I*2^(2/3)*sqrt(pi)/(gamma(5/6)*gamma(2/3)*gamma(1/3)^3), 0, -31104*I*pi/(gamma(5/6)^3*gamma(1/6)^3), -20736*I*pi/(gamma(5/6)^2*gamma(2/3)*gamma(1/3)*gamma(1/6)^2), 7776*I/(gamma(5/6)^2*gamma(1/6)^2), 0, 0, 0, -5184*I*2^(1/3)*sqrt(pi)/(gamma(5/6)*gamma(2/3)^2*gamma(1/6)^2), -7776*I/(gamma(5/6)^2*gamma(1/6)^2), -31104*I*pi/(gamma(5/6)^3*gamma(1/6)^3), -15552*I/(gamma(5/6)^2*gamma(1/6)^2), 0, 7776*I/(gamma(5/6)^2*gamma(1/6

In [ ]:
'''
Symbolic matrix for the 81 by 81 case, kind of useless at the moment
'''

s = list(var('x_%d' % i) for i in range(1,h22size+1))
sc= list(var('cx_%d' % i) for i in range(1,h22size))
l=s+sc#declaring symbolic variables, one x_i for each residue form, x4 being the maximally
#symmetric and real one, and the index c being for their complex conjugates
rho=matrix(SR,len(allh31))
for i in range(len(allh31)):
    for j in range(len(allh31)):
        for k in range(len(allh22)):
            if vector(allh31[i])+vector(allh31[j])+vector(allh22[k])==vector([4,4,4,4,4,4]):
                rho[i,j]=l[k]
print(rho)

In [41]:
'''
USED FOR THE 9 BY 9 CASE ON THE PAPER AND REQUIRES THE CONVENTION TO BE MET

'''

#computing the ranks of each period matrix at every point (rescaling) of the lattice
rankings=[]
for i in range(len(rescaling)):
    x0=rescaling[i][0]*alphas[0]
    x1=rescaling[i][1]*alphas[1]
    x2=rescaling[i][2]*alphas[2]
    x3=rescaling[i][3]*alphas[3]
    x4=rescaling[i][4]*alphas[4]
    x0c=rescaling[i][0].conjugate()*ccalphas[0]
    x1c=rescaling[i][1].conjugate()*ccalphas[1]
    x2c=rescaling[i][2].conjugate()*ccalphas[2]
    x3c=rescaling[i][3].conjugate()*ccalphas[3]
    periodmat=matrix(SR,6,[[0,0,0,x2c,0,x0c],[0,x3c,x3,x1c,0,x4],[0,x3,0,0,0,0],
                 [x2c,x1c,0,0,x3,x2],[0,0,0,x3,0,x1],[x0c,x4,0,x2,x1,x0]])
    rankings.append(periodmat.rank())

In [42]:
print(rankings) #one can check for few coordinates such as there but in general do not do it

[2]


In [43]:
#Finally compute lengths of every point in the lattice
def explore(rootlattice,putcoords):
    '''
    Allows to check if the results of the quadratic form agree with the result of the lattice by brute forcing the computation
    Extremely slow due to using symbolic variables 
    Only used in 9 by 9 case
    '''
    explored=[]
    for i in range(len(putcoords)):
        vec=vector(putcoords[i])
        explored.append(rootlattice*vec)
    summ=0
    length=[]
    for i in range(len(explored)):
        for j in range(len(explored[i])):
            if j==4: #if it's the h22 form, we need to treat it separately for INTEGRALITY
                #rework if Z+1/2 notice j=4 corresponding to the position of the (2^6) form
                #which needs to be changed if it changes index.
                #replalce by len(nocch22_even)
                summ+=4*(2*pi)^4/(2^7*3^5)*explored[i][j]*conjugate(explored[i][j])*alphas[j]*ccalphas[j]
            else:
                summ+=2*(2*pi)^4/(2^7*3^5)*explored[i][j]*conjugate(explored[i][j])*alphas[j]*ccalphas[j]
        length.append(summ)
        summ=0
    return length

In [45]:
length=explore(change_basis,coords)


In [47]:
print(length) #good result !

[243/2]


In [ ]:
#below we write it in txt files because sagemath is so slow we need to switch to python to even plot everything

In [99]:
with open("\\Users\\hugof\\Downloads\\2x2all_close_points", "w") as f:
    f.write(str(length))
with open("\\Users\\hugof\\Downloads\\2x2coords_all_close_points", "w") as g:
    g.write(str(coords))
#computations and writing the length in a txt file

In [104]:
#this is so extremely long
from sympy import *
sim_length=[]
for i in range(len(length)):
    sim_length.append(simplify(length[i]))
    
with open("\\Users\\hugof\\Downloads\\2x2simplified_lengths_all_close_points", "w") as h:
    h.write(str(sim_length))

In [47]:
with open("\\Users\\hugof\\Downloads\\ranks_all_close_points", "w") as i:
    i.write(str(rankings))